In [ ]:
!pip install beautifulSoup4
!pip install selenium
!pip install pandas
!pip install azure-storage-blob==12.8.0
!pip install multithreading

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from azure.storage.blob import BlobClient
from io import BytesIO
import pandas as pd
import multithreading
import time
import requests
import json
import math


driver_path=r'D:\chromedriver.exe'
CONNECTION_STRING="DefaultEndpointsProtocol=https;AccountName=msftadintel1483425946;AccountKey=/Cxn1abl9zkdv8y6+pLxxuMfY74YxHFdaGlL9OsKOH8J18IHOQM1pH1ULNf2vWdsInY/mvm8z/Q9jz5vUSYB5A==;EndpointSuffix=core.windows.net;"
CONTAINER_NAME="poc-2"

def initiate_driver(driver_path):
    options = Options()
    prefs = {
      "translate_whitelists": {"de":"en"},
      "translate":{"enabled":"true"}
    }
    options.add_experimental_option("prefs", prefs)
    options.add_argument("--lang=en")
    driver = webdriver.Chrome(driver_path,options=options)
    return driver

def get_william_cookies(driver):
    dcookies=driver.get_cookies()
    cookies={}
    for i in dcookies:
        cookies[i['name']]=i['value']
    return cookies

def waitForPageLoad(seconds=10):
    iter_count=int(seconds/2)
    for r in range(iter_count):
        time.sleep(2)
        if driver.execute_script('''return document.readyState''')=='complete':
            break
        else:
            time.sleep(2)
            pass
        
def convert_size(size_bytes):
    if size_bytes!='-':
        size_bytes=float(size_bytes)
        if size_bytes == 0:
            return "0 B"
        size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
        i = int(math.floor(math.log(size_bytes, 1024)))
        p = math.pow(1024, i)
        s = round(size_bytes / p, 2)
        return "%s %s" % (s, size_name[i])
    
def asset_upload(row,i,img_count,img_res_type,img_res_type_col,uploaded_assets,cookies):
    if img_count != "*":
        if i+1>img_count:
            print(f"Successfully Migrated {img_count} {img_res_type} Resolution Assets")
            return uploaded_assets
    try:
        DESTINATION_BLOB=f"williams-f1-race/photos/{row['Season']}season/{row['Collection']}/{row['Days of Week']}/{row[img_res_type_col]}"
        ASSET_NAME=row[img_res_type_col]
        print(f"{i+1}. Migrating Asset ID: {row['Asset Id']}")
        headers = {
            'authority': 'williamsracing.photos',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'accept-language': 'en-US,en;q=0.9'}

        response = requests.get(f"https://williamsracing.photos/pictures/{ASSET_NAME}", headers=headers, cookies=cookies,stream=True)

        response.raise_for_status()
        blob_client = BlobClient.from_connection_string(conn_str=CONNECTION_STRING, container_name=CONTAINER_NAME
                                                        , blob_name=DESTINATION_BLOB)
        response.raw.decode_content = True
        source_io=BytesIO(response.raw.data)
        blob_client.upload_blob(source_io, blob_type="BlockBlob",overwrite=True)
        bprop=blob_client.get_blob_properties()
        uploaded_asset_size=convert_size(bprop['size'])
        status='success'
        print(f"Uploaded Asset Size: {uploaded_asset_size}")

        uploaded_assets.append((row['Asset Id'],row[img_res_type_col],img_res_type,row[img_res_type_col.replace('Name','Size')],uploaded_asset_size,status))
        i=i+1
    except:
        try:
            print("Sleeping for 3 seconds")
            time.sleep(3)
            DESTINATION_BLOB=f"williams-f1-race/photos/{row['Season']}season/{row['Collection']}/{row['Days of Week']}/{row[img_res_type_col]}"
            ASSET_NAME=row[img_res_type_col]
            print(f"{i+1}. Retry - Migrating Asset ID: {row['Asset Id']}")
            headers = {
                'authority': 'williamsracing.photos',
                'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
                'accept-language': 'en-US,en;q=0.9'}

            response = requests.get(f"https://williamsracing.photos/pictures/{ASSET_NAME}", headers=headers, cookies=cookies,stream=True)

            response.raise_for_status()
            blob_client = BlobClient.from_connection_string(conn_str=CONNECTION_STRING, container_name=CONTAINER_NAME
                                                            , blob_name=DESTINATION_BLOB)
            response.raw.decode_content = True
            source_io=BytesIO(response.raw.data)
            blob_client.upload_blob(source_io, blob_type="BlockBlob",overwrite=True)
            bprop=blob_client.get_blob_properties()
            uploaded_asset_size=convert_size(bprop['size'])
            status='success'

            print(f"Retry - Uploaded Asset Size: {uploaded_asset_size}")
            uploaded_assets.append((row['Asset Id'],row[img_res_type_col],img_res_type,row[img_res_type_col.replace('Name','Size')],uploaded_asset_size,status))

        except Exception as e:
            print(e)
            uploaded_asset_size='-'
            status='failed'
            print(f'Failed To Migrate Asset ID: {row["Asset Id"]}')
            uploaded_assets.append((row['Asset Id'],row[img_res_type_col],img_res_type,row[img_res_type_col.replace('Name','Size')],uploaded_asset_size,status))

            #err_img.append(row['Asset Id'])
        i=i+1
    return uploaded_assets

def migrate_photo_assets_to_azure(assets_path,img_res_type,img_count="*",season='*',collection='*'):
    uploaded_assets=[]
    list_of_results = []

    df_assets=pd.read_csv(assets_path)
    df_assets['Season']=df_assets['Season'].astype(str)
    if (season !='*') and (collection != '*'):
        df_assets=df_assets.loc[(df_assets['Collection']==collection) & (df_assets['Season']==season)]
    if len(df_assets)==0:
        print("Please Provide Valid Season or Collection Name")
    else:
        if img_res_type.lower() == 'high':
            img_res_type_col = 'Hi Res Assest Name'
            
        else:
            img_res_type_col = 'Lo Res Assest Name'
        print(f"Image Resolution Type: {img_res_type}")
        print(f"Initializing To Migrate {len(df_assets)} Assets")
        # i=0

        with multithreading.Pool(multithreading.cpu_count()) as p:
            completed = p.starmap(asset_upload, [(row,_,img_count,img_res_type,img_res_type_col,uploaded_assets,cookies) for _,row in df_assets.iterrows()])
        
        list_of_results.extend([ c[0] for c in completed])
        return list_of_results
    
    
if __name__=='__main__':

    uname="williamsmaster"
    pswd="MSTR2022WF1"
    driver=initiate_driver(driver_path)
    photo_url='https://williamsracing.photos/'
    driver.get(photo_url)
    waitForPageLoad()
    # login
    # username=driver.find_element_by_id('id_username')
    username = driver.find_element(By.ID, 'id_username') 
    username.click()
    username.clear()
    username.send_keys(uname)

    # password=driver.find_element_by_id('id_password')
    password = driver.find_element(By.ID, 'id_password')
    password.click()
    password.clear()
    password.send_keys(pswd)

    # logIn=driver.find_element_by_xpath("//button[contains(text(),'Log in')]")
    logIn = driver.find_element(By.XPATH, "//button[contains(text(),'Log in')]")
    logIn.click()
    waitForPageLoad()
    cookies=get_william_cookies(driver)
    driver.quit()
    # The below function will migrate all the """low resolution"""" assets to azure
    # low --> Low Resolution Photo Assets
    # high --> High Resolution Photo Assets
               
    assets_path=r"D:\USA Code\William Race\Inventory\WR_Photo_Assets_FinalCopy.csv"
    uploaded_assets=migrate_photo_assets_to_azure(assets_path,'high',season='2022',collection='Miami GP')
    df_log=pd.DataFrame(uploaded_assets,columns=['Asset Id','Asset Name','Image Resolution Type','Asset Actual Size','Asset Migrated Blob Size','Migration Status'])
    df_log
    df_log.to_csv("XBP Photo Assets Migration Log.csv",index=False)
    

    
    